In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd




In [3]:
df = pd.read_csv('HDBCarparkInformation.csv')
display(df)

,car_park_no,address,x_coord,y_coord,car_park_type,type_of_parking_system,short_term_parking,free_parking,night_parking,car_park_decks,gantry_height,car_park_basement
0,ACB,BLK 270/271 ALBERT CENTRE BASEMENT CAR PARK,30314.7936,31490.4942,BASEMENT CAR PARK,ELECTRONIC PARKING,WHOLE DAY,NO,YES,1,1.80,Y
1,ACM,BLK 98A ALJUNIED CRESCENT,33758.4143,33695.5198,MULTI-STOREY CAR PARK,ELECTRONIC PARKING,WHOLE DAY,SUN & PH FR 7AM-10.30PM,YES,5,2.10,N
2,AH1,BLK 101 JALAN DUSUN,29257.7203,34500.3599,SURFACE CAR PARK,ELECTRONIC PARKING,WHOLE DAY,SUN & PH FR 7AM-10.30PM,YES,0,0.00,N
3,AK19,BLOCK 253 ANG MO KIO STREET 21,28185.4359,39012.6664,SURFACE CAR PARK,COUPON PARKING,7AM-7PM,NO,NO,0,0.00,N
4,AK31,BLK 302/348 ANG MO KIO STREET 31,29482.0290,38684.1754,SURFACE CAR PARK,COUPON PARKING,NO,NO,NO,0,0.00,N
...,...,...,...,...,...,...,...,...,...,...,...,...
2238,Y81M,BLK 476 YISHUN STREET 44,30194.3665,45535.3488,MULTI-STOREY CAR PARK,ELECTRONIC PARKING,WHOLE DAY,SUN & PH FR 7AM-10.30PM,YES,15,2.15,N
2239,Y82M,BLK 478 YISHUN STREET 42,29935.5818,45679.7181,MULTI-STOREY CAR PARK,ELECTRONIC PARKING,WHOLE DAY,SUN & PH FR 7AM-10.30PM,YES,11,2.15,N
2240,Y83L,BLK 382 YISHUN STREET 31,29649.6679,45882.3415,SURFACE CAR PARK,ELECTRONIC PARKING,NO,NO,NO,0,0.00,N
2241,Y83M,BLK 382 YISHUN STREET 31,29505.6858,45847.8567,MULTI-STOREY CAR PARK,ELECTRONIC PARKING,WHOLE DAY,SUN & PH FR 7AM-10.30PM,YES,16,2.15,N


In [19]:
# Bulk data must be formatted as newline-delimited JSON (NDJSON).

# The following code will generate a file named "carpark.ndjson" in the current directory.
# The file will contain the data from the DataFrame in NDJSON format.

with open('carpark.ndjson', 'w') as f:
    for _, row in df.iterrows():
        f.write(row.to_json() + '\n')
        




In [ ]:
## Python script to upload the NDJSON to elastic search
import os
from dotenv import load_dotenv
from elasticsearch import Elasticsearch, helpers
import json

# Load environment variables
load_dotenv()

# Elasticsearch credentials
username = 'elastic'
password = os.getenv('ELASTIC_PASSWORD')  # Make sure to set this environment variable
api_key = os.getenv('ELASTIC_API_KEY')  # Make sure to set this environment variable

# Connect to Elasticsearch
client = Elasticsearch(
    "http://localhost:9200",
    # basic_auth=(username, password)
    http_auth=(username, password),
    api_key=api_key
)

# Check Elasticsearch connection
# print(client.info())


{'name': '62453a56faae', 'cluster_name': 'docker-cluster', 'cluster_uuid': '6WyCFai4TOyQjCkcb1bZWA', 'version': {'number': '8.16.0', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '12ff76a92922609df4aba61a368e7adf65589749', 'build_date': '2024-11-08T10:05:56.292914697Z', 'build_snapshot': False, 'lucene_version': '9.12.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


In [27]:
resp = client.indices.create(
    index="carparks",
)
print(resp)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'carparks'}


In [28]:
def upload_ndjson_to_es(file_path, index_name):
    """
    Upload an NDJSON file to Elasticsearch.
    :param file_path: Path to the NDJSON file.
    :param index_name: Name of the Elasticsearch index to upload data to.
    """
    try:
        with open(file_path, 'r') as f:
            actions = [
                {
                    "_index": index_name,
                    "_source": json.loads(line)
                }
                for line in f
            ]
        
        # Bulk upload to Elasticsearch
        helpers.bulk(client, actions)
        print(f"Data successfully uploaded to index: {index_name}")
    except Exception as e:
        print(f"Error uploading data to Elasticsearch: {e}")

# File path and index name
ndjson_file = 'carpark.ndjson'  # Replace with your NDJSON file path
index_name = 'carpark-index'   # Replace with your desired Elasticsearch index name

# Upload the NDJSON file
upload_ndjson_to_es(ndjson_file, index_name)

Data successfully uploaded to index: carpark-index


In [ ]:
### query the first 10 records
resp = client.search(
    index="carpark-index",
)
print(resp)

NotFoundError: NotFoundError(404, 'index_not_found_exception', 'no such index [carpark_index]', carpark_index, index_or_alias)